This is the jupyter notebook for the paper called
# Which features of postural sway are effective in distinguishing Parkinson’s disease patients from controls? An experiment

In here we use postural sway data from the Canberra Hospital. We featurise this using the features from a recent systematic review: https://onlinelibrary.wiley.com/doi/10.1002/brb3.1929.

The postural sway data is first preprocessed. The features are then extracted, for which the effectiveness is calculated and compared, according to the claims made in the literature review

## Load the data and clean it

In [1]:
import plotly.io as pio
pio.renderers.default = 'notebook'

In [2]:
from package import featureHandler as fh
from package import dataHandler as dh
import pandas as pd
import numpy as np
from scipy import signal
from os import listdir
from os.path import isfile, join
import copy
import matplotlib.pyplot as plt
import itertools

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
#Get list of participant as well as their demographics
participants = pd.read_csv(join('Data','participants.csv'))
participant_information = pd.read_csv(join('Data','participant information.csv'))
participants = dh.df_retrieve(participants,{'Greenlight':1})
participants = participants.merge(participant_information,on='Participant',how='left')
is_PD = ['P' in p for p in participants['Participant']]
participants['is PD'] = is_PD

#Get the list of postural sway files I have available
path = join('Data','Postural Sway')
swayfiles = [f for f in listdir(path) if isfile(join(path, f)) and ('.mat' in f)]

#Join the list of participants with their corresponding postural sway files
participants.index = participants['Participant']
for swayfile in swayfiles:
    if swayfile[:4] in participants['Participant'].tolist(): #assumes swayfiles are named PXXX_EY.mat
        if 'eo' in swayfile.lower():
            participants.at[swayfile[:4],'EO sway file'] = swayfile
        elif 'ec' in swayfile.lower():
            participants.at[swayfile[:4],'EC sway file'] = swayfile
        else:
            print('What visual state is this?? The name is %s' % swayfile)
participants = participants[['Age','Sex','Yrs since diagnosis','UPDRS','is PD','EC sway file','EO sway file']]

#Print data summary
dh.data_summary(participants)

There are 24 PD patients, 16 males and 8 females, with age 67.00 (+- 9.06), UPDRS 25.50 (+- 13.08), and yrs since diagnosis 6.98 (+- 5.20)
	The 16 males have age 69.81 (+- 8.91), UPDRS 26.34 (+- 13.22), and yrs since diagnosis 6.92 (+- 4.73)
	The 8 females have age 60.57 (+- 5.74), UPDRS 23.57 (+- 13.58), and yrs since diagnosis 7.11 (+- 6.49)
There are 15 control participants, 5 males and 10 females, with age 72.40 (+- 10.56)
	The 5 males have age 67.60 (+- 9.18)
	The 10 females have age 72.40 (+- 10.56)


## Preprocess the data
This consists of cutting all data to be consistently 90 seconds long, scaling from metres to millimetres, lowpass filtered at 20 Hz, cutting out the first and last 15 seconds, demeaning the data, and then finally decimating the data to the desired sampling frequency.

Decimation occurs with an antialiasing filter at $f_s/2$ Hz with an 8th order Chebyshev type 1 filter

In [4]:
def process(data, lowpass_cutoff, lowpass_order, decimate_order, demean=False, scale=1):
    """Preprocess the data. This assumes the data hasnt been played with yet, i.e., fs = 1000
    A scale=1 means the units are in meters"""
    
    #If the data is 120 seconds, cut to 90 by removing last 30 seconds
    data = [d[:90000] for d in data]
    data[4] = data[4] * scale
    data[5] = data[5] * scale
    
    #Low pass filter
    sos = signal.butter(lowpass_order, lowpass_cutoff, 'lowpass', fs=1000, output='sos')
    data = [data[0]] + [signal.sosfilt(sos,d) for d in data[1:]]
    
    #Remove first and last 15 seconds, resulting in a 60 second sample
    #This also removes artifacts in the butterworth filter, as it assumes a start from 0, 0, which is not the case
    data = [d[15000:75000] for d in data]
    
    #Recenters the data if needed, but not t and Fz
    if demean: data = [data[0], data[1]] + [d - d.mean() for d in data[2:]]
    
    #Adjust time so that it starts from 0
    data[0] = data[0] - min(data[0])
    
    #Decimate
    data = dh.decimate(data,decimate_order)
    
    return data

## Extract features

The features are saved as a dictionary with the keys as ['10 Hz', '20 Hz', '40 Hz', '100 Hz']. The corresponding values are a DataFrame with columns as features and rows as participants. Note that EO and EC features from one participant is under the same row as different features (columns)


In [5]:
###############################################
overwrite = False
###############################################

data_path = join('Data','Postural Sway')

#Creating a base results DataFrame to copy when using different sampling rates
base_feature_names,visual_feature_names = fh.get_feature_names()
feature_names = list(itertools.chain(*[[name+'_EC',name+'_EO'] for name in base_feature_names])) + visual_feature_names
base_features = pd.DataFrame(columns = feature_names)

#Creating copies for the given sampling rates
decimate_orders = [[10,10], [10,5], [5,5] ,10] #10, 20, 40, 100 Hz
features = {}

if not overwrite:
    if isfile(join('Results','features.pickle')):
        features = dh.pickleLoad(path=join('Results','features.pickle'))
    else:
        print('features.pickle doesnt exist')
else:
    if isfile(join('Results','features.pickle')): print('features.pickle already exists. We are now overwriting this')
    for decimate_order in decimate_orders:
        fs = 1000/np.prod(decimate_order)
        features['%i Hz' % fs] = copy.copy(base_features)
        lowpass_cutoff = 20
        for participant in participants.index:
            eo_sway_file = participants.loc[participant,'EO sway file']
            ec_sway_file = participants.loc[participant,'EC sway file']

            if isinstance(eo_sway_file,str): #if the path is a string, i.e., not NaN
                eo_data = dh.load_sway_file(join(data_path,eo_sway_file))
                eo_data = process(eo_data,lowpass_cutoff=lowpass_cutoff,lowpass_order=4,decimate_order=decimate_order,demean=True,scale=1000)
                eo_features = fh.get_all_features(eo_data)
                eo_features = {key+'_EO':val for key,val in eo_features.items()}

            if isinstance(ec_sway_file,str): #if the path is a string, i.e., not NaN
                ec_data = dh.load_sway_file(join(data_path,ec_sway_file))
                ec_data = process(ec_data,lowpass_cutoff=lowpass_cutoff,lowpass_order=4,decimate_order=decimate_order,demean=True,scale=1000)
                ec_features = fh.get_all_features(ec_data)
                ec_features = {key+'_EC':val for key,val in ec_features.items()}

            if isinstance(eo_sway_file,str) and isinstance(ec_sway_file,str):
                eoec_features = fh.get_all_visual_features(eo_data, ec_data)

            features['%i Hz' % fs].loc[participant] = {**eo_features, **ec_features, **eoec_features} #merge dict

    #save these results
#    dh.pickleSave(path=join('Results','features.pickle'),obj=features)

## Removing perfectly correlated features
We have identified the features that are perfectly correlated in the notebook: "Perfectly correlated features"

The pairs are:
- 'area90_ML', 'rms_displacement_ML'
- 'area90_ML', 'std_displacement_ML'
- 'area90_AP', 'rms_displacement_AP'
- 'area90_AP', 'std_displacement_AP'
- 'pathlength', 'avg_velocity'
- 'pathlength', 'swayvector_length'
- 'pathlength_ML', 'avg_velocity_ML'
- 'pathlength_AP', 'avg_velocity_AP'
- 'rms_displacement', 'planardeviation'
- 'rms_displacement_ML', 'std_displacement_ML'
- 'rms_displacement_AP', 'std_displacement_AP'
- 'avg_displacement', 'avg_radius'
- 'avg_velocity', 'swayvector_length'

In [6]:
#Theres quite a few. To get rid of these perfect correlations, we remove: 
features_to_remove = ['std_displacement_ML',
                      'std_displacement_AP',
                      'planardeviation',
                      'avg_velocity_AP',
                      'avg_velocity_ML',
                      'area90_AP',
                      'area90_ML',
                      'swayvector_length',
                      'avg_radius',
                      'avg_velocity']

#Remove it from base_feature_names
for name in features_to_remove: base_feature_names.remove(name)
    
features_to_remove = [f+'_EO' for f in features_to_remove] + [f+'_EC' for f in features_to_remove] #include back EO/EC

for fs_key in features.keys():
    ind = features[fs_key].columns.isin(features_to_remove)
    features[fs_key] = features[fs_key].loc[:,~ind]

## Hypothesis: Feature effectiveness with lower $f_s$ are also lower 

This was one of the claims stated in the literature review.

To test this, we first determine the effectiveness of each feature. This was quantified in two ways:

The first way was to simply calcualted the effect size of each feature. The effect size we use here is Hedges' $g$, as it handles large differences in variances and number of samples better than other effect sizes. This is defined as 
\begin{equation}
    \frac{\overline{x}_1 - \overline{x}_2}{s^*}, \quad \textrm{where} \quad s^* = \sqrt{\frac{(n_1-1)s^2_1 + (n_2-1)s^2_2}{n_1+n_2-2}}
\end{equation}

The second way was to use machine learning. Here we perform out-of-sample testing with 10-times repeated 5-fold cross validation using a classification model. The data was randomly paired across PD and control classes by matching the sex and (roughly) the age of participants. This mean that the 39 participants were reduced to 30 for machine learning evaluations. The model we use here is an soft ensemble of an rbf-SVM, logistic regression with L2 penalty, and random forests with 100 trees. This evaluation was repeated for every single feature, resulting in an averaged AUROC which quantifies effectiveness of that feature.

The effectiveness (both ES and AUROC) of features across sampling frequencies are compared by scaling them by the maximum value across sampling frequencies, so that the effectiveness of each feature is represented as a percentage of the maximum value. This distribution of percentages for a given sampling frequency is compared to the ideal (1 with no divergence) using the Wasserstein metric, and visually compared with a kernel density estimate.


Additionally, we perform the same machine learning experiment but with multiple features. Here we use all features at once, and we use several feature selection methods. These were a wrapper method with a linear-SVM with L1 penalty, a minimum redundance method by choosing features with least correlation to each other, a maximum relevance method by choosing features most relevant to the target variable (defined with mutual information and ANOVA F-value). We also used PCA, though this isn't really a feature selection method, it reduces dimensionality and serves a similar function.

### Now looking at the effectiveness of using a subset of all features

In [7]:
from sklearn import preprocessing
from plotly import graph_objects as go
from package import resultsHandler as rh
from package import ml
import plotly.figure_factory as ff
import baycomp
from package import effect_sizes as es
from scipy.stats import wasserstein_distance
from pdf2image import convert_from_path

dpi=300

In [8]:
from sklearn.feature_selection import SelectKBest, SelectFromModel, mutual_info_classif, f_classif
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA

In [30]:
AUROCs = dh.pickleLoad(join('Results','AUROCs.pickle'))

In [ ]:
def keys():
    return ['10 Hz','20 Hz','40 Hz','100 Hz']

feature_selection_results = pd.DataFrame()
feature_selection_full_results = pd.DataFrame(columns = keys(),dtype=object)
##################################################################
repetitions=10
seeds=list(range(repetitions))
return_dataframe=True
SelectKBest_settings = list(itertools.product((f_classif, mutual_info_classif),list(range(2,24,2))))
lsvc_C_settings = (0.1, 1.0, 10.0, 0.08, 0.8, 8.0, 0.3, 3.0)
#PCA_settings = list(range(2,24))
mincorr_settings = list(range(24))
##################################################################
        
    
for func, k in SelectKBest_settings:
    
    for fs_key in keys():
        feature_selector = SelectKBest(func, k=k)

        AUROC = ml.get_all_AUROCs_with_feature_selection(participants,features[fs_key],feature_selector,repetitions,return_dataframe,seeds)
        feature_selection_full_results.at['%s, features=%i, AUROC' % (func.__name__,k), fs_key] = AUROC.T.unstack().values
        AUROC = AUROC.mean(axis=1)
        
        feature_selection_results.at['%s, features=%i, AUROC mean' % (func.__name__,k), fs_key] = AUROC.mean()
        feature_selection_results.at['%s, features=%i, AUROC std' % (func.__name__,k), fs_key] = AUROC.std()       
        #print(fs_key,k,func)

#################################

for C in lsvc_C_settings:    
    
    for fs_key in keys():        
        model = LinearSVC(C=C, penalty="l1", dual=False, max_iter=100000, random_state=0) 
        feature_selector = SelectFromModel(model, prefit=False)
        
        AUROC = ml.get_all_AUROCs_with_feature_selection(participants,features[fs_key],feature_selector,repetitions,return_dataframe,seeds)
        feature_selection_full_results.at['LinearSVC, C=%.3f, AUROC' % (C), fs_key] = AUROC.T.unstack().values
        AUROC = AUROC.mean(axis=1)

        feature_selection_results.at['LinearSVC, C=%.3f, AUROC mean' % (C), fs_key] = AUROC.mean()
        feature_selection_results.at['LinearSVC, C=%.3f, AUROC std' % (C), fs_key] = AUROC.std()       
        #print(fs_key,C)
        
#################################

#for n_components in PCA_settings:
    
#    for fs_key in features.keys():        
#        feature_selector = PCA(n_components=n_components)
        
#        AUROC = ml.get_all_AUROCs_with_feature_selection(participants,features[fs_key],feature_selector,repetitions,return_dataframe,seeds)
#        feature_selection_full_results.at['n_components=%i, AUROC' % (n_components), fs_key] = AUROC.T.unstack().values
#        AUROC = AUROC.mean(axis=1)

#        feature_selection_results.at['n_components=%i, AUROC mean' % (n_components), fs_key] = AUROC.mean()
#        feature_selection_results.at['n_components=%i, AUROC std' % (n_components), fs_key] = AUROC.std()

###################################

for fs_key in keys():
    corr=features[fs_key].corr()
    best_feature = AUROCs[fs_key].mean(axis=1).idxmax()
    all_features = [best_feature]
    
    for i in mincorr_settings:
        c = abs(corr[all_features]).sum(axis=1)
        all_features.append(c.idxmin())

        AUROC = ml.get_all_AUROCs_with_feature_selection(participants,features[fs_key][all_features],None,repetitions,True,seeds)
        feature_selection_full_results.at['mincorr num_features=%i, AUROC'%len(all_features), fs_key] = AUROC.T.unstack().values
        AUROC = AUROC.mean(axis=1)
        
        feature_selection_results.at['mincorr num_features=%i, AUROC mean'%len(all_features),fs_key] = AUROC.mean()
        feature_selection_results.at['mincorr num_features=%i, AUROC std'%len(all_features),fs_key] = AUROC.std()
        #print(fs_key,i)

###################################

print('Overall feature selection results')
feature_selection_results_summary = pd.DataFrame()
idxmax = feature_selection_results.idxmax()
kf_results_selected = {}
for fs_key in idxmax.index:
    best_method_name = idxmax[fs_key]
    feature_selection_results_summary.at['Best selection method',fs_key] = best_method_name[:-5]
    feature_selection_results_summary.at['Mean of AUROC using best selection',fs_key] = feature_selection_results.loc[best_method_name,fs_key]
    feature_selection_results_summary.at['Stdev of AUROC using best selection',fs_key] = feature_selection_results.loc[best_method_name[:-4] + 'std',fs_key]
    kf_results_selected[fs_key] = feature_selection_full_results.loc[best_method_name[:-5],fs_key]
display(feature_selection_results_summary)

Best feature selection methods:
- 10 Hz: mincorr num_features=20
- 20 Hz: mincorr num_features=2
- 40 Hz: f_classif, features=22
- 100 Hz: f_classif, features=20

In [68]:
fs_key = '10 Hz'

corr = features[fs_key].corr()
best_feature = AUROCs[fs_key].mean(axis=1).idxmax()
all_features = [best_feature]

for i in range(19):
    c = abs(corr[all_features]).sum(axis=1)
    all_features.append(c.idxmin())
best_subset_10 = all_features

#########################

fs_key = '20 Hz'

corr = features[fs_key].corr()
best_feature = AUROCs[fs_key].mean(axis=1).idxmax()
all_features = [best_feature]

c = abs(corr[all_features]).sum(axis=1)
all_features.append(c.idxmin())
best_subset_20 = all_features    
    
#########################

fs_key = '40 Hz'

func = f_classif
k=22
feature_selector = SelectKBest(func, k=k)
_ = ml.get_all_AUROCs_with_feature_selection(participants,features[fs_key],feature_selector,repetitions,return_dataframe,seeds)
best_subset_40 = features[fs_key].columns[feature_selector.get_support()]

#########################

fs_key = '100 Hz'

func = f_classif
k=20
feature_selector = SelectKBest(func, k=k)
_ = ml.get_all_AUROCs_with_feature_selection(participants,features[fs_key],feature_selector,repetitions,return_dataframe,seeds)
best_subset_100 = features[fs_key].columns[feature_selector.get_support()]


In [73]:
for p in best_subset_10:
    print(p)
    
print('\n')
for p in best_subset_20:
    print(p)
    
print('\n')
for p in best_subset_40:
    print(p)
    
print('\n')
for p in best_subset_100:
    print(p)

peak_displacement_backward_EC
DTYC_EC
DYL_EO
%recurrence_ML_EC
swaymovement_ML_EO
bandpower_4-7_ML_EC
RQA_trend_ML_EO
bandpower_2-4_ML_EO
%recurrence_ML_EO
VRI_avg_displacement_AP
%recurrence_AP_EO
area95_majoraxis_angle_EO
DTYC_EO
VRI_swayarea
%recurrence_AP_EC
bandpower_4-7_ML_EC
%recurrence_ML_EC
avg_displacement_ML_EC
VRI_avg_displacement_AP
%recurrence_ML_EO


area95_majoraxis_length_EC
%recurrence_ML_EC


area95_majoraxis_length_EC
pathlength_ML_EC
rms_displacement_EC
rms_displacement_AP_EC
displacement_range_AP_EC
peak_displacement_AP_EC
peak_displacement_forward_EC
peak_displacement_backward_EC
direction_index_ML_EC
direction_index_AP_EC
frequency95_AP_EO
frequency90_AP_EO
totalenergy_AP_EC
DTXC_EO
DTRC_EC
DTRC_EO
Y2_EC
DRS_EC
DYL_EC
fractaldimension_EC
swayvector_angle_EC
RQA_trend_ML_EC


pathlength_ML_EC
rms_displacement_AP_EC
displacement_range_AP_EC
peak_displacement_AP_EC
peak_displacement_forward_EC
peak_displacement_backward_EC
direction_index_ML_EC
direction_index_AP_E